# Chapter 02. 1차원 데이터 정리

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plot_util import plot_var_interact, plot_std_interact

# Jupyter notebook의 출력을 소수점 이하 3자리로 제한
%precision 3
# DataFrame 의 출력을 소수점 이하 3자리로 제한
pd.set_option('display.precision', 3)

%matplotlib inline

In [7]:
df = pd.read_csv('./python_stat_sample-master/data/ch2_scores_em.csv', index_col = 'student number')
df.head() # 처음 5행 표시

,english,mathematics
student number,,
1,42,65
2,69,80
3,56,63
4,41,63
5,57,76


학생 번호(student number)순 10명의 영어점수 array 형태로 저장

In [8]:
scores = np.array(df['english'])[:10]
scores

array([42, 69, 56, 41, 57, 48, 65, 49, 65, 58], dtype=int64)

학생 점수별 A~J 까지 이름 부여한 dataframe 생성

In [9]:
scores_df = pd.DataFrame({'score':scores},
                        index = pd.Index(['A','B','C','D','E','F','G','H','I','J'],
                                        name = 'student'))
scores_df

,score
student,
A,42
B,69
C,56
D,41
E,57
F,48
G,65
H,49
I,65


## 2.1 데이터 중심의 좌표

### 2.1.1 평균값(mean)

In [11]:
# 파이썬 함수 이용
sum(scores) / len(scores)

55.000

In [12]:
# numpy 함수 이용
np.mean(scores)

55.000

In [15]:
# dataframe 함수 이용 >> pandas 함수 없음 (pd.mean() - X)
scores_df.mean()

score    55.0
dtype: float64

In [17]:
# array 함수 이용
scores.mean()

55.000

### 2.1.2 중앙값(median)
- 평균값에 비해 이상값에 영향을 덜 받음
- 데이터의 개수가 짝수일 경우, 중앙값은 가운데 두 값(n/2, n/2+1)의 평균값으로 정의

In [18]:
# 크기 순으로 나열하기 위해 np.sort() 사용
sorted_scores = np.sort(scores)
sorted_scores

array([41, 42, 48, 49, 56, 57, 58, 65, 65, 69], dtype=int64)

In [19]:
n = len(sorted_scores)
if n % 2 == 0 :
    m0 = sorted_scores[n//2 - 1]
    m1 = sorted_scores[n//2]
    median = (m0 + m1)/2
else :
    median = sorted_scores[(n+1)//2 -1]

median

56.500

In [20]:
# NumPy 함수 이용
np.median(scores)

56.500

In [21]:
# dataframe 함수 이용
scores_df.median()

score    56.5
dtype: float64

### 2.1.3 최빈값(mode)
- 데이터에서 가장 많이 나타나는 값
- 기본적으로 질적 데이터의 대표값을 구할 때 사용하는 지표
    - 양적 데이터에서는 완전히 동일한 점수가 여러 번 나오는 경우가 거의 없어 유일한 값이 결정되지 않을 때가 많기 때문
    - 도수분포표를 도입하면 양적 데이터에서도 정의 가능

In [23]:
pd.Series([1,1,1,2,2,3]).mode()

0    1
dtype: int64

In [26]:
pd.Series([1,2,3,4,5]).mode()

0    1
1    2
2    3
3    4
4    5
dtype: int64

## 2.2 데이터의 산포도 지표

### 2.2.1 분산과 표준편차

#### - 편차(deviation)
- 각 데이터가 평균으로부터 어느 정도 떨어져 있는가를 나타내는 지표

NumPy 에는 브로드캐스트*라는 기능이 있으므로 다음과 같이 편차 계산 가능
- 브로드캐스트 : 배열과 하나의 숫자와의 조합으로 이루어진 산술 연산을 수행하는 기능
- 숫자 하나와 배열의 원소별 계산이 각각 한 번씩 수행

In [24]:
mean = np.mean(scores)
deviation = scores - mean
deviation

array([-13.,  14.,   1., -14.,   2.,  -7.,  10.,  -6.,  10.,   3.])

In [25]:
another_scores = [50, 60, 58, 54, 51, 56, 57, 53, 52, 59]
another_mean   = np.mean(another_scores)
another_deviation = another_scores - another_mean
another_deviation

array([-5.,  5.,  3., -1., -4.,  1.,  2., -2., -3.,  4.])

In [27]:
np.mean(deviation)

0.000

In [28]:
np.mean(another_deviation)

0.000

편차의 평균은 항상 0이므로 두가지의 경우를 편차 평균으로 비교 불가

In [29]:
summary_df = scores_df.copy()
summary_df['deviation'] = deviation
summary_df

,score,deviation
student,,
A,42,-13.0
B,69,14.0
C,56,1.0
D,41,-14.0
E,57,2.0
F,48,-7.0
G,65,10.0
H,49,-6.0
I,65,10.0


In [30]:
summary_df.mean()

score        55.0
deviation     0.0
dtype: float64

#### - 분산(variance)
- 편차의 제곱의 평균

In [31]:
np.mean(deviation ** 2)

86.000

In [32]:
# NumPy 함수 이용
np.var(scores)

86.000

DataFrame, Series 에도 분산 계산하는 var메서드가 있으나, 값이 다름
- NumPy에 기본으로 설정된 분산 : 표본분산
- Pandas에 기본으로 설정된 분산 : 불편분산 (추측통계에서 매우 중요한 역할을 하는 지표이나 까다로움.. 10장에서!)
    - pandas 로 표본분산을 계산하고 싶다면 var 메서드의 인수를 ddof=0 으로 설정 (default : ddof=1 >> 불편분산)
    - 분산 계산할 때는 항상 ddof 인수 설정하여 어떤 분산 계산하는지 명시하는 것이 better!

In [33]:
scores_df.var()

score    95.556
dtype: float64

In [34]:
scores_df.var(ddof=0)

score    86.0
dtype: float64

In [36]:
# 편차 제곱 열 추가
summary_df['square of deviation'] = np.square(deviation)
summary_df

,score,deviation,square of deviation
student,,,
A,42,-13.0,169.0
B,69,14.0,196.0
C,56,1.0,1.0
D,41,-14.0,196.0
E,57,2.0,4.0
F,48,-7.0,49.0
G,65,10.0,100.0
H,49,-6.0,36.0
I,65,10.0,100.0


In [37]:
summary_df.mean()

score                  55.0
deviation               0.0
square of deviation    86.0
dtype: float64

그림 2.3 분산

In [65]:
plot_var_interact(scores[:4])

interactive(children=(IntSlider(value=42, description='A', min=1), IntSlider(value=69, description='B', min=1)…